## Data Import

In [1]:
import pandas as pd
import re
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm, tqdm_notebook, trange
tqdm.pandas()
import pickle

In [4]:
# use_col_post = ['category','content_id','user_id','title','tags','post_date','last_update','content']
# df_post = pd.read_csv('../src/pantip_car_post.csv',usecols=use_col_post)

# use_col_cmt = ['category','content_id','user_id','comment','tags','post_date','last_update']
# df_cmt = pd.read_csv('../src/pantip_car_comment.csv',usecols=use_col_cmt)

In [14]:
df_post = pd.read_csv('../src/pantip_aviation.csv')
df_post = df_post.iloc[:,[5,6,9,11,12,15,16,17]]
df_post.columns = ['category','content_id','user_id','title','tags','post_date','last_update','content']

In [3]:
# major car manufacturer
# Honda: CRV ,BRV, CIVIC, CITY, JAZZ, ACCORD
# Toyota: CHR, ALTIS, YARIS, VIOUS, CAMRY

In [15]:
df_post.head()

,category,content_id,user_id,title,tags,post_date,last_update,content
0,travel,38338139,342269.0,แลกไมล์ Lufthansa,"สะสมไมล์,สายการบิน",2018-12-06 14:22:26,0000-00-00 00:00:00,สอบถามเพื่อนๆว่าไมล์สะสมของ Lufthansa แลกอะไรไ...
1,pantip_chalermthai,37876053,3982877.0,Moon 2009 ทำไมพระเอกถึงคุยกับแคทรีนได้ครับถ้าเ...,"The Moon That Embraces the Sun,Interstellar (ภ...",2018-07-18 11:44:12,2018-07-18 12:15:09,ทำไมพระเอกคุยกับภรรยาได้ครับทั้งที่ตัวเองเป็นโ...
2,pantip_chalermthai,37712061,4571934.0,ทำไมบริษัทใหญ่แนวหน้าของไทย..ทำการตลาดร่วมกับ....,"Tesco Lotus,การบินไทย,Starbucks,SF Cinema,Tops...",2018-05-28 02:27:11,2018-05-28 03:19:42,แค่งงว่าบริษัทใหญ่ๆทำการตลาด..เป็นPartner กับแ...
3,pantip_chalermkrung,37767545,2821989.0,ทำไมการบินไทยไม่เปิดเพลง 'ปลายทางคือคุณ' ในไฟล...,"เพลงไทยสากล,การบินไทย,เพลงประกอบโฆษณา",2018-06-13 18:26:00,0000-00-00 00:00:00,พอดีมีโอกาสได้เห็นเพลง 'ปลายทางคือคุณ' ที่คุณว...
4,pantip_chalermthai,37324175,3960437.0,ขอถามคนที่เป็นนักบินหน่อยค่ะ,"งานสายการบิน,ภาพยนตร์ต่างประเทศ",2018-01-28 15:41:24,0000-00-00 00:00:00,คือเราดูหนังมีฉากเครื่องบินตกบลาๆๆๆ และมีลำนึง...


In [16]:
df_post.shape

(54872, 8)

In [17]:
df_post['title-content'] = df_post['title'].fillna('') + ' ' + df_post['content'].fillna('')

In [19]:
# randomly check the text
rand_topic_content = df_post.loc[np.random.randint(0,df_post.shape[0]),'title-content']
print(rand_topic_content)

nokair เปลี่ยนเวลาบิน ต้องทำอย่างไรดีคะ พอดีดิฉันจองเที่ยวบินกับนกแอร์จาก ลำปาง-ดอนเมือง ของวันเสาร์ที่ 9 มกราคม 2559รหัสการจอง 5197xxxxดิฉันไม่ได้เอะใจจนกระทั่งมาเช็คตารางเที่ยวบิน พบว่านกแอร์ได้แจ้งเปลี่ยนเวลาบินจากเดิมบิน 14.20น.เปลี่ยนเป็นเริ่มบิน 15.05น.ซึ่งเวลาต่างคือ 45 นาที ดิฉันจึงอยากถามว่าเรามีสิทธิ์ที่จะเปลี่ยนเที่ยวบินได้รึเปล่าคะ หรือทางสายการบินจะคืนเงินให้เรารึเปล่าดิฉันจึงมาตั้งกระทู้ถามค่ะ รบกวนผู้รู้ช่วยตอบที ขอบคุณค่ะ


## Text cleaning

In [20]:
# prepare token from pantip data
import string
import re

# put everything all together as a function
def clean_msg(msg):
    
    # remove all the html notation---OK
    msg = re.sub( r"(\<(\/)?(\w)*(\d)?\>)",'', msg)
    
    # remove the link---OK
    msg = re.sub(r'https*.*? ',' url ',msg)
    
    # remove the link2---OK
    msg = re.sub(r'https*.*',' url ',msg)

    # remove the link3---OK
    msg = re.sub(r'www*.*? ',' url ',msg)
    
    # remove the link4---OK
    msg = re.sub(r'www*.*',' url ',msg)
    
    # remove the email---OK
    msg = re.sub(r'[\w.-]+@[\w.]+',' email ',msg)
    
    # remove the date---OK
    msg = re.sub(r'\d+\W\d+\W\d+','date ',msg)

    # remove the date2---OK
    msg = re.sub(r'\d+\W\d+\W\d+',' date ',msg)
    
    # remove the time <--- need to check logic
    msg = re.sub(r'\d+\s*:\s*\d+',' time ',msg)
    
    # remove the time2 <--- need to check logic
    msg = re.sub(r'\d+\s*\.\s*\d+',' time ',msg)
    
    # remove the number
    msg = re.sub(r' \d+,*\d*,*\d*,*\d*,*\d*,*\d* ',' number ',msg)
    
    # removing punctuation ---> move to the dataframe function
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),' ',msg)
        
    # remove white blank space
    msg = re.sub(r'\s+',' ',msg)
    
    return msg

In [21]:
df_post['title_content_clean'] = df_post['title-content'].progress_apply(clean_msg)

100%|██████████| 54872/54872 [00:11<00:00, 4667.99it/s]


In [22]:
df_post = df_post.dropna(subset=['post_date','content_id'])
df_post = df_post.reset_index(drop=True)

In [12]:
# df_post['content_id'].progress_apply(lambda x:re.sub(r'\d+','',x) == '')

## Text tokenizing

In [23]:
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords

thai_stop = thai_stopwords()

def tokenize(text):
    
    # tokenize
    tokens = word_tokenize(text,'ulmfit')
    
    # remove stopwords
    tokens = [tkn for tkn in tokens if not tkn in thai_stop]
    
    # change to lower case to normalise English word
    tokens = [tkn.lower() for tkn in tokens]
    
    return tokens

In [24]:
tokens_post = []
for i in trange(df_post.shape[0]):
    tokens_post.append(tokenize(df_post.iloc[i]['title_content_clean']))

100%|██████████| 54857/54857 [02:36<00:00, 350.57it/s]


In [25]:
# drop list with zero token
cnt_tkn = [len(l) for l in tokens_post]
cnt_tkn_serie = pd.Series(cnt_tkn)

index_drp = cnt_tkn_serie[cnt_tkn_serie==0].index.values

# drop original topic from top_df
df_post.drop(index_drp,inplace=True)

# drop from input list
for index in sorted(index_drp, reverse=True):
    del tokens_post[index]

In [26]:
print(len(tokens_post), len(df_post))
index_drp = df_post.loc[df_post.post_date.apply(len)!=19].index.values

# drop original topic from top_df
df_post.drop(index_drp,inplace=True)

# drop from input list
for index in sorted(index_drp, reverse=True):
    del tokens_post[index]
    
print(len(tokens_post), len(df_post))

54857 54857
54844 54844


In [27]:
df_post['post_date'] = pd.to_datetime(df_post['post_date'],errors='coerce')

print(len(tokens_post), len(df_post))
index_drp = df_post.loc[df_post.post_date.isna()].index.values

# drop original topic from top_df
df_post.drop(index_drp,inplace=True)

# drop from input list
for index in sorted(index_drp, reverse=True):
    del tokens_post[index]
    
print(len(tokens_post), len(df_post))

54844 54844
54844 54844


In [28]:
df_post.post_date.describe()

count                   54844
unique                  54832
top       2014-10-10 11:00:39
freq                        2
first     2013-01-01 19:55:37
last      2018-12-10 23:27:23
Name: post_date, dtype: object

In [29]:
with open('../out/aviation/tokens_post.txt', 'wb') as fp:
    pickle.dump(tokens_post, fp)
    
with open('../out/aviation/tokens_post.txt', 'rb') as fp:
    tokens_post = pickle.load(fp)

In [30]:
df_post.to_csv('../out/aviation/df_post.csv',index=False)

df_post = pd.read_csv('../out/aviation/df_post.csv')